This Tutorial is a modified version of a SageMaker Tutorial. This was modified to show how to integrate Arize into SageMaker.

# Tutorial Overview
This tutorial goes through the following steps:
1. Create a realtime ETL pipeline with SparkML loaded as a SageMaker Model
2. Serialize the model with MLeap
3. Create an XGBoost Model 
4. Deploy both the ETL Model and XGBoost into a pipeline model
5. Deploy and Test to an Endpoint
6. Create a Lambda Function that integrates ARIZE AI and invokes Sagemaker
7. Test the Lambda function
8. Create a Gateway to internet that triggers the Lambda function 
9. Call the prediction service

# Feature processing with Spark, training with XGBoost and deploying as Inference Pipeline

Typically a Machine Learning (ML) process consists of few steps: gathering data with various ETL jobs, pre-processing the data, featurizing the dataset by incorporating standard techniques or prior knowledge, and finally training an ML model using an algorithm.

In many cases, when the trained model is used for processing real time or batch prediction requests, the model receives data in a format which needs to pre-processed (e.g. featurized) before it can be passed to the algorithm. In the following notebook, we will demonstrate how you can build your ML Pipeline leveraging Spark Feature Transformers and SageMaker XGBoost algorithm & after the model is trained, deploy the Pipeline (Feature Transformer and XGBoost) as an Inference Pipeline behind a single Endpoint for real-time inference and for batch inferences using Amazon SageMaker Batch Transform.

In this notebook, we use Amazon Glue to run serverless Spark. Though the notebook demonstrates the end-to-end flow on a small dataset, the setup can be seamlessly used to scale to larger datasets.

## Objective: predict the age of an Abalone from its physical measurement

The dataset is available from [UCI Machine Learning](https://archive.ics.uci.edu/ml/datasets/abalone). The aim for this task is to determine age of an Abalone (a kind of shellfish) from its physical measurements. At the core, it's a regression problem. The dataset contains several features - `sex` (categorical), `length` (continuous), `diameter` (continuous), `height` (continuous), `whole_weight` (continuous), `shucked_weight` (continuous), `viscera_weight` (continuous), `shell_weight` (continuous) and `rings` (integer).Our goal is to predict the variable `rings` which is a good approximation for age (age is `rings` + 1.5). 

We'll use SparkML to process the dataset (apply one or many feature transformers) and upload the transformed dataset to S3 so that it can be used for training with XGBoost.

## Methodologies
The Notebook consists of a few high-level steps:

* Using SageMaker XGBoost to train on the processed dataset produced by SparkML job.
* Building an Inference Pipeline consisting of SparkML & XGBoost models for a realtime inference endpoint.
* Building an Inference Pipeline consisting of SparkML & XGBoost models for a single Batch Transform job.
* Integrating the Model with Arize

#  AWS Glue 

* The original tutorial used AWS Glue. This tutorial was modified so the entire tutorial runs in this Notebook. No need to use AWS Glue.

### Finding out the current execution role of the Notebook
We are using SageMaker Python SDK to retrieve the current role for this Notebook which needs to be enhanced.

In [ ]:
import sagemaker
import os
from sagemaker import get_execution_role
sess = sagemaker.Session()
role = get_execution_role()
print(role[role.rfind('/') + 1:])

In [ ]:
!pip install pyspark==2.2.3

### Creating an S3 bucket and uploading this dataset
Next we will create an S3 bucket with the `aws-glue` string in the name and upload this data to the S3 bucket. In case you want to use some existing bucket to run your Spark job via AWS Glue, you can use that bucket to upload your data provided the `Role` has access permission to upload and download from that bucket.

Once the bucket is created, the following cell would also update the `abalone.csv` file downloaded locally to this bucket under the `input/abalone` prefix.

In [ ]:
import boto3
import botocore
from botocore.exceptions import ClientError

boto_session = sess.boto_session
s3 = boto_session.resource('s3')
account = boto_session.client('sts').get_caller_identity()['Account']
region = boto_session.region_name
default_bucket = 'aws-glue-{}-{}'.format(account, region)

try:
    if region == 'us-east-1':
        s3.create_bucket(Bucket=default_bucket)
    else:
        s3.create_bucket(Bucket=default_bucket, CreateBucketConfiguration={'LocationConstraint': region})
except ClientError as e:
    error_code = e.response['Error']['Code']
    message = e.response['Error']['Message']
    if error_code == 'BucketAlreadyOwnedByYou':
        print ('A bucket with the same name already exists in your account - using the same bucket.')
        pass        

# Uploading the training data to S3
sess.upload_data(path='abalone.csv', bucket=default_bucket, key_prefix='input/abalone')

## Downloading dataset and uploading to S3
SageMaker team has downloaded the dataset from UCI and uploaded to one of the S3 buckets in our account. In this Notebook, we will download from that bucket and upload to your bucket so that AWS Glue can access the data. The default AWS Glue permissions we just added expects the data to be present in a bucket with the string `aws-glue`. Hence, after we download the dataset, we will create an S3 bucket in your account with a valid name and then upload the data to S3. 

In [ ]:
!wget https://s3-us-west-2.amazonaws.com/sparkml-mleap/data/abalone/abalone.csv


## Writing the feature processing script using SparkML

The code for feature transformation using SparkML is found below it uses standard SparkML constructs to define the Pipeline for featurizing the data.

Once the Spark ML Pipeline `fit` and `transform` is done, we are splitting our dataset into 80-20 train & validation as part of the script and uploading to S3 so that it can be used with XGBoost for training.

### Upload MLeap dependencies to S3

For our job, we will also have to pass MLeap dependencies to Glue. MLeap is an additional library we are using which does not come bundled with default Spark.

Similar to most of the packages in the Spark ecosystem, MLeap is also implemented as a Scala package with a front-end wrapper written in Python so that it can be used from PySpark. We need to make sure that the MLeap Python library as well as the JAR is available within the Glue job environment. In the following cell, we will download the MLeap Python dependency & JAR from a SageMaker hosted bucket and upload to the S3 bucket we created above in your account. 

If you are using some other Python libraries like `nltk` in your code, you need to download the wheel file from PyPI and upload to S3 in the same way. At this point, Glue only supports passing pure Python libraries in this way (e.g. you can not pass `Pandas` or `OpenCV`). However you can use `NumPy` & `SciPy` without having to pass these as packages because these are pre-installed in the Glue environment. 

In [ ]:
!wget https://s3-us-west-2.amazonaws.com/sparkml-mleap/0.9.6/python/python.zip
!wget https://s3-us-west-2.amazonaws.com/sparkml-mleap/0.9.6/jar/mleap_spark_assembly.jar

## Defining output locations for the data and model
Next we define the output location where the transformed dataset should be uploaded. We are also specifying a model location where the MLeap serialized model would be updated. 

By designing our code in that way, we can re-use these variables as part of other SageMaker operations from this Notebook (details below).

In [ ]:
python_dep_location = sess.upload_data(path='python.zip', bucket=default_bucket, key_prefix='dependencies/python')
jar_dep_location = sess.upload_data(path='mleap_spark_assembly.jar', bucket=default_bucket, key_prefix='dependencies/jar')

In [ ]:
out_arg = '--jars ' + jar_dep_location + ' pyspark-shell'
#os.environ['PYSPARK_SUBMIT_ARGS'] = out_arg

In [ ]:

from time import gmtime, strftime
import time

timestamp_prefix = strftime("%Y-%m-%d-%H-%M-%S", gmtime())

# Input location of the data, We uploaded our train.csv file to input key previously
s3_input_bucket = default_bucket
s3_input_key_prefix = 'input/abalone'

# Output location of the data. The input data will be split, transformed, and 
# uploaded to output/train and output/validation
s3_output_bucket = default_bucket
s3_output_key_prefix = timestamp_prefix + '/abalone'

# the MLeap serialized SparkML model will be uploaded to output/mleap
s3_model_bucket = default_bucket
s3_model_key_prefix = s3_output_key_prefix + '/mleap'

In [ ]:
args = {}
args.update({'S3_INPUT_BUCKET' :s3_input_bucket,'S3_INPUT_KEY_PREFIX':s3_input_key_prefix,'S3_OUTPUT_BUCKET':s3_output_bucket,
                                         'S3_OUTPUT_KEY_PREFIX':s3_output_key_prefix, 
                                         'S3_MODEL_BUCKET':s3_model_bucket,
                                         'S3_MODEL_KEY_PREFIX':s3_model_key_prefix})

In [ ]:
from __future__ import print_function
from __future__ import unicode_literals


In [ ]:
import time
import sys
import os
import shutil
import csv
import boto3

In [ ]:
!pip install mleap

## This is the SparkML pipeline portion of the script
We start a spark sesssion, build the pipeline, define the schema, process the training data.

Lastly a MLeap spark model gets created for a searlized call in SageMaker Pipeline.

In [ ]:

import pyspark
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.sql.types import StructField, StructType, StringType, DoubleType
from pyspark.ml.feature import StringIndexer, VectorIndexer, OneHotEncoder, VectorAssembler
from pyspark.sql.functions import *
from mleap.pyspark.spark_support import SimpleSparkSerializer

In [ ]:
def csv_line(data):
    r = ','.join(str(d) for d in data[1])
    return str(data[0]) + "," + r

In [ ]:
spark = SparkSession.builder.config("spark.jars", "mleap_spark_assembly.jar").appName("PySparkAbalone").getOrCreate()


In [ ]:
!wget https://s3-us-west-2.amazonaws.com/sparkml-mleap/data/abalone/abalone.csv


In [ ]:
spark.sparkContext._jsc.hadoopConfiguration().set("mapred.output.committer.class",
                                                      "org.apache.hadoop.mapred.FileOutputCommitter")

### Schema From File
Define schema of data from file



In [ ]:
columns = ["sex", "length", "diameter", "height","whole_weight","shucked_weight","viscera_weight","shell_weight"]
pred_column = "rings"

In [ ]:
schema = StructType([StructField(columns[0], StringType(), True), 
                         StructField(columns[1], DoubleType(), True),
                         StructField(columns[2], DoubleType(), True),
                         StructField(columns[3], DoubleType(), True),
                         StructField(columns[4], DoubleType(), True),
                         StructField(columns[5], DoubleType(), True),
                         StructField(columns[6], DoubleType(), True), 
                         StructField(columns[7], DoubleType(), True), 
                         StructField(pred_column, DoubleType(), True)])

In [ ]:
total_df = spark.read.csv('./abalone.csv', header=False, schema=schema)

### DataPipeLine Transforms
One Hot Encoder and String indexer are used to convert the Sex (Male/Female) column to a 1 hot feature


In [ ]:
sex_indexer = StringIndexer(inputCol="sex", outputCol="indexed_sex")

In [ ]:
sex_encoder = OneHotEncoder(inputCol="indexed_sex", outputCol="sex_vec")

In [ ]:
assembler = VectorAssembler(inputCols=["sex_vec", 
                                           "length", 
                                           "diameter", 
                                           "height", 
                                           "whole_weight", 
                                           "shucked_weight", 
                                           "viscera_weight", 
                                           "shell_weight"], 
                                outputCol="features")

In [ ]:
pipeline = Pipeline(stages=[sex_indexer, sex_encoder, assembler])

This creates the Data Transform by calling Fit. No Model is present yet / this is only data transforms

In [ ]:
model = pipeline.fit(total_df)

In [ ]:
transformed_total_df = model.transform(total_df)

In [ ]:
total_df.show()

In [ ]:
transformed_total_df.show()

In [ ]:
(train_df, validation_df) = transformed_total_df.randomSplit([0.8, 0.2])

In [ ]:
transformed_total_df.select("features").show(30,False)

In [ ]:
train_rdd = train_df.rdd.map(lambda x: (x.rings, x.features))

In [ ]:
train_lines = train_rdd.map(csv_line)

The train and validation data sets are stored to files for Train and Validation of SageMaker pipeline.

In [ ]:
train_lines.saveAsTextFile('./' + args['S3_OUTPUT_KEY_PREFIX'] + 'train')

In [ ]:
validation_rdd = validation_df.rdd.map(lambda x: (x.rings, x.features))

In [ ]:
validation_lines = validation_rdd.map(csv_line)

In [ ]:
validation_lines.saveAsTextFile('./' + args['S3_OUTPUT_KEY_PREFIX'] + 'validate')

In [ ]:
!rm -rf /home/ec2-user/SageMaker/modelnew.zip

### MLeap
This searlizes the model into a executable that is used by the SageMaker Pipeline. You can think of the datatransformation as now encapsulated in the serialized model. This model will be one part of the sagemaker pipeline.

In [ ]:
SimpleSparkSerializer().serializeToBundle(model, "jar:file:/home/ec2-user/SageMaker/modelnew.zip", validation_df)

In [ ]:
import zipfile
with zipfile.ZipFile("/home/ec2-user/SageMaker/modelnew.zip") as zf:
    zf.extractall("/home/ec2-user/SageMaker/modelnew")

import tarfile
with tarfile.open("/home/ec2-user/SageMaker/modelnew.tar.gz", "w:gz") as tar:
    tar.add("/home/ec2-user/SageMaker/modelnew/bundle.json", arcname='bundle.json')
    tar.add("/home/ec2-user/SageMaker/modelnew/root", arcname='root')


In [ ]:

from sagemaker.model import Model
from sagemaker.pipeline import PipelineModel
from sagemaker.sparkml.model import SparkMLModel

In [ ]:
import json
schema = {
    "input": [
        {
            "name": "sex",
            "type": "string"
        }, 
        {
            "name": "length",
            "type": "double"
        }, 
        {
            "name": "diameter",
            "type": "double"
        }, 
        {
            "name": "height",
            "type": "double"
        }, 
        {
            "name": "whole_weight",
            "type": "double"
        }, 
        {
            "name": "shucked_weight",
            "type": "double"
        },
        {
            "name": "viscera_weight",
            "type": "double"
        }, 
        {
            "name": "shell_weight",
            "type": "double"
        }, 
    ],
    "output": 
        {
            "name": "features",
            "type": "double",
            "struct": "vector"
        }
}
schema_json = json.dumps(schema)

In [ ]:
sparkml_data= sess.upload_data(path='modelnew.tar.gz', bucket=args['S3_OUTPUT_BUCKET'], key_prefix=args['S3_OUTPUT_KEY_PREFIX'])

In [ ]:
sparkml_model = SparkMLModel(model_data=sparkml_data, env={'SAGEMAKER_SPARKML_SCHEMA' : schema_json})

In [ ]:
sparkml_model.model_data

## Using SageMaker XGBoost to train on the processed dataset produced by SparkML job

Now we will use SageMaker XGBoost algorithm to train on this dataset. We already know the S3 location
where the preprocessed training data was uploaded as part of the Glue job.

### We need to retrieve the XGBoost algorithm image
We will retrieve the XGBoost built-in algorithm image so that it can leveraged for the training job.

In [ ]:
from sagemaker.amazon.amazon_estimator import get_image_uri

training_image = get_image_uri(sess.boto_region_name, 'xgboost', repo_version="latest")
print (training_image)

In [ ]:
os.path.join(args['S3_OUTPUT_BUCKET'])

### Next XGBoost model parameters and dataset details will be set properly
We have parameterized this Notebook so that the same data location which was used in the PySpark  can now be passed to XGBoost Estimator as well.

In [ ]:
s3_train_data = sess.upload_data(path='train.csv', bucket=args['S3_OUTPUT_BUCKET'], key_prefix=args['S3_OUTPUT_KEY_PREFIX'])
s3_validation_data = sess.upload_data(path='validate.csv', bucket=args['S3_OUTPUT_BUCKET'], key_prefix=args['S3_OUTPUT_KEY_PREFIX'])


In [ ]:
s3_output_location = 's3://{}/{}/{}'.format(s3_output_bucket, s3_output_key_prefix, 'xgboost_model')

In [ ]:
xgb_model_train = sagemaker.estimator.Estimator(training_image,
                                         role, 
                                         train_instance_count=1, 
                                         train_instance_type='ml.m4.xlarge',
                                         train_volume_size = 20,
                                         train_max_run = 3600,
                                         input_mode= 'File',
                                         output_path=s3_output_location,
                                         sagemaker_session=sess)

In [ ]:
xgb_model_train.set_hyperparameters(objective = "reg:linear",
                              eta = .2,
                              gamma = 4,
                              max_depth = 5,
                              num_round = 10,
                              subsample = 0.7,
                              silent = 0,
                              min_child_weight = 6)

In [ ]:
train_data = sagemaker.session.s3_input(s3_train_data, distribution='FullyReplicated', 
                        content_type='text/csv', s3_data_type='S3Prefix')
validation_data = sagemaker.session.s3_input(s3_validation_data, distribution='FullyReplicated', 
                             content_type='text/csv', s3_data_type='S3Prefix')

data_channels = {'train': train_data, 'validation': validation_data}

### Finally XGBoost training will be performed.

In [ ]:
xgb_model_train.fit(inputs=data_channels, logs=True)

# Building an Inference Pipeline consisting of SparkML & XGBoost models for a realtime inference endpoint

Next we will proceed with deploying the models in SageMaker to create an Inference Pipeline. You can create an Inference Pipeline with upto five containers.

Deploying a model in SageMaker requires two components:

* Docker image residing in ECR.
* Model artifacts residing in S3.

**SparkML**

For SparkML, Docker image for MLeap based SparkML serving is provided by SageMaker team. For more information on this, please see [SageMaker SparkML Serving](https://github.com/aws/sagemaker-sparkml-serving-container). MLeap serialized SparkML model was uploaded to S3 as part of the SparkML job we executed in AWS Glue.

**XGBoost**

For XGBoost, we will use the same Docker image we used for training. The model artifacts for XGBoost was uploaded as part of the training job we just ran.

### Passing the schema of the payload via environment variable
SparkML serving container needs to know the schema of the request that'll be passed to it while calling the `predict` method. In order to alleviate the pain of not having to pass the schema with every request, `sagemaker-sparkml-serving` allows you to pass it via an environment variable while creating the model definitions. This schema definition will be required in our next step for creating a model.

We will see later that you can overwrite this schema on a per request basis by passing it as part of the individual request payload as well.

In [ ]:
import json
schema = {
    "input": [
        {
            "name": "sex",
            "type": "string"
        }, 
        {
            "name": "length",
            "type": "double"
        }, 
        {
            "name": "diameter",
            "type": "double"
        }, 
        {
            "name": "height",
            "type": "double"
        }, 
        {
            "name": "whole_weight",
            "type": "double"
        }, 
        {
            "name": "shucked_weight",
            "type": "double"
        },
        {
            "name": "viscera_weight",
            "type": "double"
        }, 
        {
            "name": "shell_weight",
            "type": "double"
        }, 
    ],
    "output": 
        {
            "name": "features",
            "type": "double",
            "struct": "vector"
        }
}
schema_json = json.dumps(schema)
print(schema_json)


### Creating a `PipelineModel` which comprises of the SparkML and XGBoost model in the right order

Next we'll create a SageMaker `PipelineModel` with SparkML and XGBoost.The `PipelineModel` will ensure that both the containers get deployed behind a single API endpoint in the correct order. The same model would later be used for Batch Transform as well to ensure that a single job is sufficient to do prediction against the Pipeline. 

Here, during the `Model` creation for SparkML, we will pass the schema definition that we built in the previous cell.

In [ ]:
xgb_model = Model(model_data=xgb_model_train.model_data, image=training_image,role=role)


In [ ]:
#endpoint_name = 'xgb-test-debug' + timestamp_prefix + '11'
#xgb_model.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge', endpoint_name=endpoint_name)

In [ ]:
model_name = 'inference-pipeline-' + timestamp_prefix

In [ ]:

sm_model = PipelineModel(name=model_name, role=role, models=[sparkml_model, xgb_model])

### Deploying the `PipelineModel` to an endpoint for realtime inference
Next we will deploy the model we just created with the `deploy()` method to start an inference endpoint and we will send some requests to the endpoint to verify that it works as expected.

In [ ]:
endpoint_name = 'inference-pipeline-ep-' + timestamp_prefix + '11'
sm_model.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge', endpoint_name=endpoint_name)

### Invoking the newly created inference endpoint with a payload to transform the data
Now we will invoke the endpoint with a valid payload that SageMaker SparkML Serving can recognize. There are three ways in which input payload can be passed to the request:

* Pass it as a valid CSV string. In this case, the schema passed via the environment variable will be used to determine the schema. For CSV format, every column in the input has to be a basic datatype (e.g. int, double, string) and it can not be a Spark `Array` or `Vector`.

* Pass it as a valid JSON string. In this case as well, the schema passed via the environment variable will be used to infer the schema. With JSON format, every column in the input can be a basic datatype or a Spark `Vector` or `Array` provided that the corresponding entry in the schema mentions the correct value.

* Pass the request in JSON format along with the schema and the data. In this case, the schema passed in the payload will take precedence over the one passed via the environment variable (if any).

#### Passing the payload in CSV format
We will first see how the payload can be passed to the endpoint in CSV format.

In [ ]:
from sagemaker.predictor import json_serializer, csv_serializer, json_deserializer, RealTimePredictor
from sagemaker.content_types import CONTENT_TYPE_CSV, CONTENT_TYPE_JSON

In [ ]:
payload = {"data": ["F",0.515,0.425,0.14,0.766,0.304,0.1725,0.255]}

predictor = RealTimePredictor(endpoint=endpoint_name, sagemaker_session=sess, serializer=json_serializer,
                                deserializer=json_deserializer,
                                content_type='application/json', accept=CONTENT_TYPE_JSON)
print(predictor.predict(payload))

We have an example of a realtime prediction.

### Now Lets install Arize AI


In [ ]:
!pip install arize

## Testing Arize in Jupyter Notebook Environment

This will show a very simple test of logging data to Arize in the Jupyter Notebook environment. The following section will show how to deploy arize to a lambda function that represents a production environment.

In [ ]:
API_KEY_ARIZE = "INSERT YOUR KEY"

In [ ]:
from arize.api import Client
arize = Client(space_key='space_key', api_key=API_KEY_ARIZE)
labels = {}
for index, col in enumerate(columns):
    labels[col] = str(payload[index])
prediciton_result = str(predictor.predict(payload))
# Unique ID based on time - This works only if you don't want to connect prediction back
prediction_id = datetime.now().strftime('%Y%m-%d%H-%M%S-') + str(uuid4())
#LOG TO ARIZE
arize.log(model_id='sage-maker-lambda', model_version='v0.1', prediction_ids=prediction_id, prediction_labels=response_data, features=labels, actual_labels=None)

The above code shows a log in a sample environment from a Jupyter notebook.

## Testing Arize in A Production Environment

In order to test arize in a produciton environment we are going to create a Lambda function that will call the Endpoint through an API. The lambda_function.py implements the Lambda. Open the local file and replace the API_KEY_ARIZE in the Lambda_function.py


The Lambda_function.py file needs to be uploaded to S3. Since the file calls the Arize library the Arize library needs to be included with the Lambda_function.py in a Zip. This ZIP will be referenced by Lambda as the file to execute when called. 

CONFIRM THAT YOU HAVE EDITED THE API_KEY_ARIZE KEY IN THE FILE lambda_function.py BEFORE UPLOADING

THe below commands will take the local file, create a zip and then upload the zip to S3

In [ ]:
!rm -r lambda_pkg
!rm myArizeDeploymentPackage.zip

In [ ]:
!mkdir lambda_pkg
!pip install arize -t ./lambda_pkg/
!cp lambda_function.py ./lambda_pkg/lambda_function.py
%cd lambda_pkg
!zip -r ../myArizeDeploymentPackage.zip *
%cd ..

In [ ]:
batch_input_loc = sess.upload_data(path='myArizeDeploymentPackage.zip', bucket=default_bucket, key_prefix='lambda_dir_3')

In [ ]:
print(batch_input_loc)

The below can be inserted into the Lambda Test portion of the platform

In [ ]:
{
  "body": {
    "data": [
      "F",
      0.515,
      0.425,
      0.14,
      0.766,
      0.304,
      0.1725,
      0.255
    ]
  }
}

## Setting up A Lambda 
This section goes through how to setup a Lambda function to handle the requests to the endpoint.

Click the create lambada function.

<img src="https://storage.googleapis.com/arize-assets/tutorials/sagemaker/pipeline_plus_lambda/step1.png">

<img src="https://storage.googleapis.com/arize-assets/tutorials/sagemaker/pipeline_plus_lambda/step2.png">

### Select author from scratch

<img src="https://storage.googleapis.com/arize-assets/tutorials/sagemaker/pipeline_plus_lambda/step3.png">

### Code Entry Type is file from S3
Enter the URL of the s3 file below

In [ ]:
print(batch_input_loc)

### Then set the permissions 
This lambda needs to be able to invoke SageMaker and needs permission setup to do that.

<img src="https://storage.googleapis.com/arize-assets/tutorials/sagemaker/pipeline_plus_lambda/step4.png">

### Click add inline policy to the right

<img src="https://storage.googleapis.com/arize-assets/tutorials/sagemaker/pipeline_plus_lambda/step5.png">

### Select Sage Maker as Service, Action Invoke Endpoint, Resources AllResources

Permissions will show up as below

<img src="https://storage.googleapis.com/arize-assets/tutorials/sagemaker/pipeline_plus_lambda/step6.png">

## After setting up the permissions you can test the Lambda
The below json code can be entered in the <test> button to test the Lambda

In [ ]:
{
  "body": {
    "data": [
      "F",
      0.515,
      0.425,
      0.14,
      0.766,
      0.304,
      0.1725,
      0.255
    ]
  }
}

<img  src="https://storage.googleapis.com/arize-assets/tutorials/sagemaker/pipeline_plus_lambda/step7.png">

### Connect to Public Internet by setting up a Gateway
Click the API Gateway button in the Lambda function. Click the <+ Add Trigger>

<img  src="https://storage.googleapis.com/arize-assets/tutorials/sagemaker/pipeline_plus_lambda/step8.png">

<img src="https://storage.googleapis.com/arize-assets/tutorials/sagemaker/pipeline_plus_lambda/step9.png">

<img src="https://storage.googleapis.com/arize-assets/tutorials/sagemaker/pipeline_plus_lambda/step10.png">

##  Test the end point by Hitting the URL
Copy the 'URL Endpoint' in blue above

curl -H "Content-Type: application/json" --location --request POST 'URL Endpoint' --data-raw '{"data": ["F",0.515,0.425,0.14,0.766,0.304,0.1725,0.255]}'